# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

### Setup API key and Initiate Ollama and GPT client

In [ ]:
# set up environment

load_dotenv(override=True)
api_key=os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError('OPENAI_API_KEY not set. Populate .env or set the environment variable.')
open_gpt_ai=OpenAI(api_key=api_key,base_url='https://api.openai.com/v1')
ollama_ai=OpenAI(api_key=os.getenv('LLAMA_API_KEY'), base_url='http://localhost:11434/v1')
if not open_gpt_ai or not ollama_ai:
    raise ValueError('Something wrong with GPT AI or Ollama AI')

# here is the question; type over this to ask something new

system_prompt="""
You are a technical expert assistant that can answer any question about the coding questions, architecture and design. 
You can teach a complex question easily with help of many examples or analogies.
You can also use humor and wittiness to make the explanation more engaging and interesting.
Do not use any markdown formatting or longer explanation unless necessary. 
Always use  code samples to showcase various functions and use cases
You can use below example as references:

Question: What is "for book in books" mean in python?
Answer: It is a for loop which iterates over the list. Example ['book1_name', 'book2_name'] . The loop will iterate from left to right.  


Question: What is "books.sort(reverse=True)" mean in python?
Answer: It is a python function which can reverse the list , You can give custom sort keys as needed. Example: books.sort(key=lambda b: b[:1]) picks the sort key as first letter and sort it.
"""

### Creates helper function for user prompting 

In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""
def get_user_prompt(question):
    return f"""
    Question: {question}
    """
def messages_for_personal_tutor(question):
    return [{
        "role":"system", "content":system_prompt
    },{
        "role":"user","content": get_user_prompt(question)
    }]

### Creates GPT helper method for answering questions. We have used stream with help of Stream=True

- We also have used Markdown and display_handle methods of IPython.display module.
- Error handling incase of network error or something wrong happens with try /catch block

In [ ]:
# Get gpt-4o-mini to answer, with streaming
def get_apt_answer(question):       
    stream_chunks=open_gpt_ai.chat.completions.create(model=MODEL_GPT, messages=messages_for_personal_tutor(question), stream=True)
    result=""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream_chunks:
        try:
            content = chunk.choices[0].delta.content or ''
            if content:
                result += content
                update_display(Markdown(result), display_id=display_handle.display_id)
        except Exception as e:
            print("Streaming error:", e)
            break
        

### Ollama use case
- Show cases usage of ollama, Idea is to make our system prompt good enough to have same/similar answer from GPT
- Doesnt have try/catch since ollama runs on local
- Uses markdown and stream 
- Great for speed and cost saving

In [ ]:
# Get Llama 3.2 to answer
def get_llama_answer(question):
    # print(ollama_ai.chat.completions.create(model=MODEL_LLAMA, messages=[{"role":"user", "content":f"Hello {question}"}]).choices[0].message.content)       
    stream_chunks=ollama_ai.chat.completions.create(model=MODEL_LLAMA, messages=messages_for_personal_tutor(question), stream=True)
    result=""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream_chunks:
        if chunk:
            result+= chunk.choices[0].delta.content or ''
            update_display( Markdown(result),display_id=display_handle.display_id)

In [ ]:
get_llama_answer("What is deque?")

In [ ]:
get_apt_answer("What is deque?")